Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

Upload data

In [3]:
from google.colab import files
uploaded = files.upload()

Saving blue_data.csv to blue_data.csv
Saving green_data.csv to green_data.csv
Saving padparadscha_data.csv to padparadscha_data.csv
Saving pink_data.csv to pink_data.csv
Saving purple_data.csv to purple_data.csv
Saving ruby_data.csv to ruby_data.csv
Saving white_data.csv to white_data.csv
Saving yellow_data.csv to yellow_data.csv


Read csv

In [4]:
blue_data = pd.read_csv("blue_data.csv")
green_data = pd.read_csv("green_data.csv")
padparadscha_data = pd.read_csv("padparadscha_data.csv")
pink_data = pd.read_csv("pink_data.csv")
purple_data = pd.read_csv("purple_data.csv")
ruby_data = pd.read_csv("ruby_data.csv")
white_data = pd.read_csv("white_data.csv")
yellow_data = pd.read_csv("yellow_data.csv")

In [5]:
data = [blue_data, green_data, padparadscha_data, pink_data, purple_data, ruby_data, white_data, yellow_data]
data_names = ["blue_data", "green_data", "padparadscha_data", "pink_data", "purple_data", "ruby_data", "white_data", "yellow_data"]

Feature Engineering

In [10]:
# STRATIFIES K-FOLD CROSS VALIDATION

# Import Required Modules.
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import xgboost
from sklearn import datasets
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder

count = 0
for dat in data:
    y_data=dat['total_price']
    X_data=dat.drop(['total_price'],axis=1)


    # Create KFold object.
    kf = KFold(n_splits=10, shuffle=True, random_state=36)
    r2_test = []
    r2_train = []

    for train_index, test_index in kf.split(X_data, y_data):
        # Use `.iloc` to properly select rows by position
        X_train_fold = X_data.iloc[train_index].copy()
        X_test_fold = X_data.iloc[test_index].copy()
        y_train_fold = y_data.iloc[train_index].copy()
        y_test_fold = y_data.iloc[test_index].copy()

        # Handle ordinal encoding
        #oe = OrdinalEncoder(categories=[['Poor', 'Fair', 'Good','Very Good','Excellent']])
        #X_train_fold['cut_quality_encoded'] = oe.fit_transform(X_train_fold[['cut_quality']])
        #X_test_fold['cut_quality_encoded'] = oe.fit_transform(X_test_fold[['cut_quality']])



        # Handle target
        shape_means = X_train_fold.groupby('shape')['price_per_carat'].mean()
        origin_means = X_train_fold.groupby('origin')['price_per_carat'].mean()
        color_means = X_train_fold.groupby('color')['price_per_carat'].mean()
        color_intensity_means = X_train_fold.groupby('color_intensity')['price_per_carat'].mean()
        clarity_means = X_train_fold.groupby('clarity')['price_per_carat'].mean()
        treatment_means = X_train_fold.groupby('treatment')['price_per_carat'].mean()
        cut_means = X_train_fold.groupby('cut')['price_per_carat'].mean()
        cut_quality_means = X_train_fold.groupby('cut_quality')['price_per_carat'].mean()

        # Map target encoding to the training data
        X_train_fold['shape_encoded'] = X_train_fold['shape'].map(shape_means)
        X_train_fold['origin_encoded'] = X_train_fold['origin'].map(origin_means)
        X_train_fold['color_encoded'] = X_train_fold['color'].map(color_means)
        X_train_fold['color_intensity_encoded'] = X_train_fold['color_intensity'].map(color_intensity_means)
        X_train_fold['clarity_encoded'] = X_train_fold['clarity'].map(clarity_means)
        X_train_fold['cut_encoded'] = X_train_fold['cut'].map(cut_means)
        X_train_fold['treatment_encoded'] = X_train_fold['treatment'].map(treatment_means)
        X_train_fold['cut_quality_encoded'] = X_train_fold['cut_quality'].map(cut_quality_means)

        # Map target encoding to the test data
        # Handle unseen categories by filling with global mean
        global_mean = X_train_fold['price_per_carat'].mean()
        X_test_fold['shape_encoded'] = X_test_fold['shape'].map(shape_means).fillna(global_mean)
        X_test_fold['origin_encoded'] = X_test_fold['origin'].map(origin_means).fillna(global_mean)
        X_test_fold['color_encoded'] = X_test_fold['color'].map(color_means).fillna(global_mean)
        X_test_fold['color_intensity_encoded'] = X_test_fold['color_intensity'].map(color_intensity_means).fillna(global_mean)
        X_test_fold['clarity_encoded'] = X_test_fold['clarity'].map(clarity_means).fillna(global_mean)
        X_test_fold['cut_encoded'] = X_test_fold['cut'].map(cut_means).fillna(global_mean)
        X_test_fold['treatment_encoded'] = X_test_fold['treatment'].map(treatment_means).fillna(global_mean)
        X_test_fold['cut_quality_encoded'] = X_test_fold['cut_quality'].map(cut_quality_means).fillna(global_mean)



        X_train_fold = X_train_fold.drop(columns=['color', 'shape', 'color_intensity', 'origin', 'cut', 'treatment', 'clarity','price_per_carat','cut_quality'])
        X_test_fold = X_test_fold.drop(columns=['color', 'shape', 'color_intensity', 'origin', 'cut', 'treatment', 'clarity','price_per_carat','cut_quality'])
        #print(X_train_fold.dtypes)

        # Train the model.
        rfr = RandomForestRegressor(random_state=42)
        rfr.fit(X_train_fold, y_train_fold)

        # Predictions
        y_pred_fold=rfr.predict(X_test_fold)

        # Evaluate the model
        r2 = r2_score(y_test_fold,y_pred_fold)
        r2_test.append(r2)

        # Check overfitting
        y_pred_train_fold=rfr.predict(X_train_fold)

        train_r2=r2_score(y_train_fold, y_pred_train_fold)
        r2_train.append(train_r2)



    # Print the output.
    print(data_names[count])
    print("\n")
    print('List of possible accuracy for test:', r2_test)
    print('\nMaximum Testing Accuracy That can be obtained from this model is:',
      max(r2_test)*100, '%')
    print('\nMinimum Testing Accuracy:',
      min(r2_test)*100, '%')
    print('\nOverall Testing Accuracy:',
      mean(r2_test)*100, '%')
    print('\nStandard Deviation is:', stdev(r2_test))

    # Print the output.
    print('List of possible accuracy for train:', r2_train)
    print('\nMaximum Training Accuracy That can be obtained from this model is:',
      max(r2_train)*100, '%')
    print('\nMinimum Training Training Accuracy:',
      min(r2_train)*100, '%')
    print('\nOverall Training Accuracy:',
      mean(r2_train)*100, '%')
    print('\nStandard Deviation is:', stdev(r2_train))
    count +=1
    print("---------------------------------------------------------------------\n")


blue_data


List of possible accuracy for test: [0.21143462247369815, 0.7424777659965378, 0.6304066794391217, 0.7451414959704972, 0.9356399941003261, 0.7970551324677831, 0.8090125152143062, 0.8874978505848786, 0.4536746794431211, 0.898220378366545]

Maximum Testing Accuracy That can be obtained from this model is: 93.56399941003261 %

Minimum Testing Accuracy: 21.143462247369815 %

Overall Testing Accuracy: 71.10561114056814 %

Standard Deviation is: 0.22561315407005234
List of possible accuracy for train: [0.9559742169466967, 0.9464404155817702, 0.9526895727300081, 0.9526093954879279, 0.9573663543182604, 0.9564629432044546, 0.9527662567189336, 0.9507836737745936, 0.9789027069758409, 0.9582170985512686]

Maximum Training Accuracy That can be obtained from this model is: 97.89027069758409 %

Minimum Training Training Accuracy: 94.64404155817702 %

Overall Training Accuracy: 95.62212634289754 %

Standard Deviation is: 0.008708551224520161
-------------------------------------------------